In [10]:
from typing import Iterator, List
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTPage

def read_file(file_name):
    file = extract_pages(file_name)
    return file
    
    
def get_paragraphs(file: Iterator[LTPage]) -> List[dict]:
    paragraphs = []
    page_num = 0
    paragraph_id = 0
    for page_layout in file:
        page_num += 1
        
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                paragraphs.append({
                    'id': paragraph_id,
                    'page': page_num,
                    'paragraph': element.get_text()
                })
                paragraph_id += 1
                
    return paragraphs






in_file = read_file('../data/raw/Chapter-1---The-Rise-of-Platform-Ecosystems_2014_Platform-Ecosystems.pdf')
paragraphs = get_paragraphs(in_file)

for i in range(10):
    print(f"{i + 1}: {paragraphs[i]['paragraph']}\n\n")



<generator object extract_pages at 0x109a178b0>